In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install yt_dlp
!pip install ffmpeg
!mkdir raw-video
!mkdir compressed-video
!mkdir log

mkdir: cannot create directory ‘raw-video’: File exists
mkdir: cannot create directory ‘compressed-video’: File exists
mkdir: cannot create directory ‘log’: File exists


In [2]:
#@title Imports all libraries
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

from googleapiclient.http import MediaFileUpload

import yt_dlp
import ffmpeg
import subprocess
import sys


In [3]:
#@title Config Authorize Google Drive API
# If modifying these scopes, delete the file token.json.
USE_SERVICE_ACCOUNT = False
SCOPES = ["https://www.googleapis.com/auth/drive"]
TOKEN_FILE = "token.json"
CREDENTIALS_FILE = "credentials.json"
FOLDER_ID = "1-3y0hToO4S8KhoWzlrqcGwjOsBWOKvoX"

In [4]:
url = "https://jrjr7jgjnmnz.cdn-centaurus.com/v/01/00096/zn4yg8o3ypqw_o/SSPD-101_javhd_today.mp4?t=TiYiagTjPWDLLyf7tc6Fjzu9If1-9MPIeiB0rSYcLzI&amp;s=1744826855&amp;e=129600&amp;f=11739256&amp;sp=400&amp;i=0.0"

In [5]:
output_filename = 'SSPD-101'

# Specify the output path
output_path = f'raw-video/'

# Create YoutubeDL object with the desired options
ydl_opts = {
  'outtmpl': output_path + '%(title)s.%(ext)s',
}

def download_from_url(url):
    info_dict = ydl.extract_info(url, download=True)
    
    # Extract the actual file name from the template
    downloaded_file_name = info_dict['title'] + '.' + info_dict['ext']
    
    return downloaded_file_name

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    output_filename = download_from_url(url)
    
    print(output_filename)

[generic] Extracting URL: https://jrjr7jgjnmnz.cdn-centaurus.com/v/01/00096/zn4yg8o3ypqw_o/SSPD-101_javhd_today.mp4?t=TiYia...amp;sp=400&amp;i=0.0
[generic] SSPD-101_javhd_today.mp4?t=TiYiagTjPWDLLyf7tc6Fjzu9If1-9MPIeiB0rSYcLzI&amp;s=1744826855&amp;e=129600&amp;f=11739256&amp;sp=400&amp;i=0: Downloading webpage


ERROR: [generic] Unable to download webpage: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)


DownloadError: ERROR: [generic] Unable to download webpage: HTTP Error 403: Forbidden (caused by <HTTPError 403: Forbidden>)

In [34]:
#@title Compress video
input_file = f'raw-video/{output_filename}'
output_file = f'compressed-video/{output_filename}'
progress_file = f'log/{output_filename}.txt'

# Set the desired video bitrate and quality (adjust as needed)
video_bitrate = '1000k'  # 1000k is just an example, you can adjust based on your requirements
video_quality = '31' # Range: 2 (best) - 31 (worst)
resolution = {
    '1080p':'1920:1080',
    '720p':'720:480',
    '480p':'640:480',
    '360p':'480:360',
    '240p':'426:240',
    '144p':'256:144'
} 

# FFmpeg command to compress the video using H.265 codec
ffmpeg_cmd = f'ffmpeg -i {input_file} -vf scale={resolution["480p"]} -b:v {video_bitrate} -vcodec libx265 -crf {video_quality} -preset veryfast -progress {progress_file} {output_file}'

# Run the FFmpeg command
subprocess.run(ffmpeg_cmd.split(), capture_output=True, text=True)


CompletedProcess(args=['ffmpeg', '-i', 'raw-video/Free_Test_Data_15MB_MP4.mp4.mp4', '-vf', 'scale=640:480', '-b:v', '1000k', '-vcodec', 'libx265', '-crf', '31', '-preset', 'veryfast', '-progress', 'log/Free_Test_Data_15MB_MP4.mp4.mp4.txt', 'compressed-video/Free_Test_Data_15MB_MP4.mp4.mp4'], returncode=0, stdout='', stderr='ffmpeg version 6.1-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers\n  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)\n  configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enabl

In [7]:
#@title Authorize Google Drive API
creds = None
if USE_SERVICE_ACCOUNT:
    creds = service_account.Credentials.from_service_account_file(
        "service_account.json", scopes=SCOPES
    )
else:
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())
service = build("drive", "v3", credentials=creds)

In [38]:
#@title List files
try:
    results = (
        service.files()
        .list(q=f"'{FOLDER_ID}' in parents and trashed = false", pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])
    if not items:
        print("No files found.")
    else:
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
except HttpError as error:
    print(f"An error occurred: {error}")

Files:
snis-228_Mosaic.mp4 (1-Gksc1nqmUdfR1ER7HBcu0pgEqNkiB9Z)
Free_Test_Data_15MB_MP4.mp4 (1-3goBUpthgyqVtFVFs_xc5aFmgapkDqL)


In [9]:
#@title copy local video to google drive

local_video_path = f'compressed-video/{output_filename}'
file_metadata = {"name": output_filename, "parents": [FOLDER_ID]}
media = MediaFileUpload(local_video_path, mimetype="video/mp4")
try:
    pass
    file = (
        service.files()
        .create(body=file_metadata, media_body=media, fields="id")
        .execute()
    )

    print(f"File ID: {file.get('id')}")
except HttpError as error:
    print(f"An error occurred: {error}")

FileNotFoundError: [Errno 2] No such file or directory: 'raw-video/$statue.mp4'